# Export and Embed plots

## Exporting plots

### PNG Generation
Bokeh can generate RGBA-format Portable Network Graphics (PNG) images from layouts using the export() function. This functionality uses a headless browser called WebKit to render the layout in memory and then capture a screenshot. The generated image will be of the same dimensions as the source layout.

#### Additional dependencies
In order to use the export() function, users have to install some additional dependencies. These dependencies can be installed via conda or pip:

`conda install selenium phantomjs pillow`

`pip install selenium pillow`

Alternatively, you can install phantomjs from npm via

`npm install -g phantomjs-prebuilt`


Note that the minimum compatible version of PhantomJS is 2.1, regardless of how it is installed.

### Example usage

In [1]:
from bokeh.plotting import figure, output_notebook, show

# prepare some data
x = [1, 2, 3, 4, 5]
y = [6, 7, 2, 4, 5]

# create a new plot with a title and axis labels
p = figure(title="simple line example", x_axis_label='x', y_axis_label='y')

# add a line renderer with legend and line thickness
p.line(x, y, legend="Temp.", line_width=2)

# show the results
output_notebook()
show(p)

Loading BokehJS ...

In [2]:
from bokeh.io import export_png

export_png(p, filename="plot.png")

'/Users/pamaron/Code/bokeh-data-visualisation/plot.png'

**Change the output backend**

In [4]:
p.output_backend = "svg"

In [7]:
from bokeh.io import export_svgs

export_svgs(p, filename="plot.svg")

['plot.svg']

## Embedding plots

### Embedding Plots and Apps
Bokeh provides a variety of ways to embed plots and data into HTML documents. First, a reminder of the distinction between standalone documents and apps:


#### Standalone Documents
These are Bokeh documents that are not backed by a Bokeh server. They may have many tools and interactions (e.g. from CustomJS callbacks) but are self-contained HTML, JavaScript, and CSS. They can be embedded into other HTML pages as one large document, or as a set of sub-components templated individually.
#### Bokeh Applications
These are Bokeh documents that are backed by a Bokeh Server. In addition to all the features of standalone documents, it is also possible to connect events and tools to real Python callbacks, to execute that execute in the Bokeh server. See Running a Bokeh Server for more information about creating and running Bokeh apps.

### Standalone Documents
This section describes how Bokeh standalone documents (i.e. those that are not linked to a Bokeh server) may be published or embedded in a variety of ways.

#### HTML files
Bokeh can generate complete HTML pages for Bokeh documents using the file_html() function. This function can emit HTML from its own generic template, or a template you provide. These files contain the data for the plot inline and are completely transportable, while still providing interactive tools (pan, zoom, etc.) for your plot. Here is an example:

In [8]:
from bokeh.plotting import figure
from bokeh.resources import CDN
from bokeh.embed import file_html

plot = figure()
plot.circle([1,2], [3,4])

html = file_html(plot, CDN, "my plot")

In [13]:
print(html[:100])





<!DOCTYPE html>
<html lang="en">
  
  <head>
    
      <meta charset="utf-8">
      <title>my p


The returned HTML text can be saved to a file using standard python file operations. You can also provide your own template and pass in custom, or additional, template variables. See the file_html() documentation for more details.

### Components

It is also possible to ask Bokeh to return the individual components of a standalone document for individual embedding using the `components()` function. This function returns a `<script>` that contains the data for your plot, together with an accompanying `<div>` tag that the plot view is loaded into. These tags can be used in HTML documents however you like:

In [14]:
from bokeh.plotting import figure
from bokeh.embed import components

plot = figure()
plot.circle([1,2], [3,4])

script, div = components(plot)

In [ ]:
print(script)

In [ ]:
print(div)

These two elements can be inserted or templated into your HTML text, and the script, when executed, will replace the div with the plot.

Using these components assumes that BokehJS has already been loaded, for instance either inline in the document text, or from CDN. To load BokehJS from CDN, add the following lines in your HTML text or template with the appropriate version replacing x.y.z:

```html
<link
    href="https://cdn.pydata.org/bokeh/release/bokeh-x.y.z.min.css"
    rel="stylesheet" type="text/css">
<link
    href="https://cdn.pydata.org/bokeh/release/bokeh-widgets-x.y.z.min.css"
    rel="stylesheet" type="text/css">
<link
    href="https://cdn.pydata.org/bokeh/release/bokeh-tables-x.y.z.min.css"
    rel="stylesheet" type="text/css">

<script src="https://cdn.pydata.org/bokeh/release/bokeh-x.y.z.min.js"></script>
<script src="https://cdn.pydata.org/bokeh/release/bokeh-widgets-x.y.z.min.js"></script>
<script src="https://cdn.pydata.org/bokeh/release/bokeh-tables-x.y.z.min.js"></script>
``` 

The "-widgets" files are only necessary if your document includes Bokeh widgets. Similarly, the "-tables" files are only necessary if you are using Bokeh data tables in your document.

For example, to use version 0.12.13, including widgets and tables support:

```html
<link
    href="https://cdn.pydata.org/bokeh/release/bokeh-0.12.13.min.css"
    rel="stylesheet" type="text/css">
<link
    href="https://cdn.pydata.org/bokeh/release/bokeh-widgets-0.12.13.min.css"
    rel="stylesheet" type="text/css">
<link
    href="https://cdn.pydata.org/bokeh/release/bokeh-tables-0.12.13.min.css"
    rel="stylesheet" type="text/css">

<script src="https://cdn.pydata.org/bokeh/release/bokeh-0.12.13.min.js"></script>
<script src="https://cdn.pydata.org/bokeh/release/bokeh-widgets-0.12.13.min.js"></script>
<script src="https://cdn.pydata.org/bokeh/release/bokeh-tables-0.12.13.min.js"></script>
```

### Autoload Scripts
A final way to embed standalone documents is the autoload_static() function. This function with provide a `<script>` tag that will replace itself with a Bokeh plot, wherever the tag happens to be located. The script will also check f or BokehJS and load it, if necessary. Using this function it is possible to embed a plot by placing this script tag alone in your document.

This function takes a Bokeh model (e.g. a plot) that you want to display, a Resources object, and a path to load a script from. Then autoload_static() will return a self-contained `<script>` tag, and a block of JavaScript code. The JavaScript code should be saved to the path you provided. The `<script>` tag, when it is included in a page, will load and run the saved JavaScript in order to realize your plot in the browser.

Here is how you might use `autoload_static()` with a simple plot:

In [17]:
from bokeh.resources import CDN
from bokeh.plotting import figure
from bokeh.embed import autoload_static

plot = figure()
plot.circle([1,2], [3,4])

js, tag = autoload_static(plot, CDN, "some/path")